Quora Insincere Question _data augmentation by replacing with similar words

In [0]:
CSV_PATH = './data/'

In [3]:
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Activation, GRU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras import activations

In [0]:
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold

In [6]:
import nltk
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
import gensim.downloader as api

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
word_vectors = api.load("glove-wiki-gigaword-100")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
word_vectors.most_similar(positive=['woman', 'king'], topn=2)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('father', 0.8199654221534729), ('mother', 0.8136025071144104)]

In [7]:
#This code snippet is inspired by https://github.com/chakki-works/chakin
from six.moves.urllib.request import urlretrieve
from progressbar import Bar, ETA, FileTransferSpeed, ProgressBar, Percentage, RotatingMarker


ROOT_DIR = './'
url = 'http://nlp.stanford.edu/data/glove.twitter.27B.zip'
file_name = url.split('/')[-1]
save_path = os.path.join(ROOT_DIR, file_name)

widgets = ['Test: ', Percentage(), ' ', Bar(marker=RotatingMarker()), ' ', ETA(), ' ', FileTransferSpeed()]
pbar = ProgressBar(widgets=widgets)

def dlProgress(count, blockSize, totalSize):
    if pbar.max_value is None:
        pbar.max_value = totalSize
        pbar.start()

    pbar.update(min(count * blockSize, totalSize))

path, _ = urlretrieve(url, save_path, reporthook=dlProgress)
pbar.finish()

Test: 100% ||                                      | Time:  0:11:42   2.1 MiB/s


In [8]:
!unzip ./glove.twitter.27B.zip

Archive:  ./glove.twitter.27B.zip
  inflating: glove.twitter.27B.25d.txt  
  inflating: glove.twitter.27B.50d.txt  
  inflating: glove.twitter.27B.100d.txt  
  inflating: glove.twitter.27B.200d.txt  


In [0]:
def load_glove_simple(word_dict):#word_index {'word': index} dictionary
    EMBEDDING_FILE = './glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = word_dict
    nb_words = len(word_index)+1
    

    embedding_matrix = np.zeros((nb_words, embed_size),dtype=np.float32) - 1.
    
    for key, i in tqdm(word_index.items()):
        word = key
        embedding_vector = embeddings_index.get(word)        
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = key.lower()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = key.upper()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        word = key.capitalize()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
            
    del embeddings_index
    import gc
    gc.collect()
    return embedding_matrix, nb_words

In [9]:
train_df = pd.read_csv(os.path.join(CSV_PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(CSV_PATH, 'test.csv'))
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (375806, 2)


In [0]:
train_X = train_df["question_text"].fillna(" ")
test_X = test_df["question_text"].fillna(" ")
train_y = train_df['target'].values
bool_train_labels = train_y != 0
#text_list = pd.concat([train_X,test_X]).apply(lambda x: x.split())

In [0]:
max_features = 95000
max_len = 100

In [0]:
def create_dict(neg_vocab,embedding_model,max_iter=None, threshold=0.5,topn=1):
    dict={}
    if max_iter is not None:
        it = min(len(neg_vocab), max_iter)
        for idx, word  in tqdm(enumerate(neg_vocab)):
            if idx > it:
                break
      
            try:
                word = word.lower()
                embedding_model[word]
                pos = pos_tag([word])
                replacements = embedding_model.most_similar(positive = word, topn=topn)
                replacements = [r for r in replacements if r[1]>threshold ]
                replacements = [r for r in replacements if pos_tag([r[0]])[0][1] == pos[0][1]]
                dict.update({word:replacements}) if len(replacements)>0 else dict
            except:
                pass
    else:
        for word  in tqdm(neg_vocab):
            try:
                word = word.lower()
                embedding_model[word]
                pos = pos_tag([word])
                replacements = embedding_model.most_similar(positive = word, topn=topn)
                replacements = [r for r in replacements if r[1]>threshold ]
                replacements = [r for r in replacements if pos_tag([r[0]])[0][1] == pos[0][1]]
                dict.update({word:replacements}) if len(replacements)>0 else dict
            except:
                pass
        
    return dict

In [0]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_df[bool_train_labels].question_text))

In [0]:
positive_sample_vocab = tokenizer.word_index

In [15]:
augment_vocab = create_dict(positive_sample_vocab, word_vectors, topn=3)

  0%|          | 0/42512 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 42512/42512 [12:38<00:00, 56.02it/s]


In [0]:
import json
json = json.dumps(augment_vocab)
f = open("./augment_dict.json","w")
f.write(json)
f.close()

In [0]:
positive_samples = list(train_df[bool_train_labels].question_text)

In [0]:
import random

def augment_sentence(augmented_dict, sentence):
    aug=[]
    for word in sentence:
        if word in augmented_dict:
            replace = augmented_dict[word][0]
            if replace[1] > random.random():
                aug.append(replace[0])
            else:
                aug.append(word)
        else:
            aug.append(word)
            
    return ' '.join(aug)

In [29]:
def augment(sentences, augmented_dict):
    augmented=[]
    for sen in tqdm(sentences):
        aug = augment_sentence(augmented_dict, sen.split())
        augmented.append(aug)
    
    return augmented

augmented_sentences = augment(positive_samples, augment_vocab)

100%|██████████| 80810/80810 [00:00<00:00, 135595.07it/s]


In [0]:
positive_labels = [1]*len(augmented_sentences)

augmented_df = pd.DataFrame(list(zip(augmented_sentences,positive_labels)), columns=['question_text', 'target'])

In [33]:
augmented_df.head()

,question_text,target
0,Has this United States become this biggest dic...,1
1,Which infants are more sweeter to their parent...,1
2,If whites support college choice and requireme...,1
3,I am gay girl and I love your nephew (boy). He...,1
4,Which races have this smallest penis?,1


In [0]:
train_df = pd.concat([train_df, augmented_df], join='inner', ignore_index=True)

In [0]:
text_list = pd.concat([train_X,test_X]).apply(lambda x: x.split())

In [0]:
train_df, val_df = train_test_split(train_df, test_size=0.08, stratify=train_df['target'] ,random_state=2018)

In [0]:

#train_X = np.array(train_df.pop('question_text')) --> This might help to save memory
train_X = train_df['question_text'].fillna(" ")
val_X = val_df['question_text'].fillna(" ")
test_X = test_df['question_text'].fillna(" ")

train_y = train_df['target'].values
val_y = val_df['target'].values

bool_train_labels = train_y != 0

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(text_list))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [0]:
train_X_pad = pad_sequences(train_X, maxlen=100,padding='post')
val_X_pad = pad_sequences(val_X, maxlen=100, padding='post')

In [0]:
dataset_v1_train = tf.data.Dataset.from_tensor_slices((train_X_pad, train_y)).shuffle(10000).batch(256)
dataset_v1_val = tf.data.Dataset.from_tensor_slices((val_X_pad, val_y)).shuffle(10000).batch(256)

In [0]:
def data_generator():
    stop = len(train_X)
    i = 0
    while i < stop:
        yield np.array(train_X[i]), np.array(train_y[i])
        i += 1
    
dataset_v2 = tf.data.Dataset.from_generator(data_generator, output_types=(tf.int32,tf.int32), output_shapes=((None), ()))
padded_dataset_v2 = dataset_v2.padded_batch(16, padded_shapes=((None,), ()))

In [0]:
a = np.arange(1,21,1).reshape(4,5)

In [0]:
choice = np.random.random(a.shape[0])< 0.5
a[choice] = a[choice] + 100

In [55]:
a

array([[101, 102, 103, 104, 105],
       [106, 107, 108, 109, 110],
       [ 11,  12,  13,  14,  15],
       [216, 217, 218, 219, 220]])